In [5]:
import numpy as np
import pandas as pd
import sqlite3


conn = sqlite3.connect("Data/database.sqlite")
cur = conn.cursor()

In [9]:
result = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]

In [18]:
newline_indent='\n'

result = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = sorted(list(zip(*result))[0])
print ("\ntables are:"+newline_indent+newline_indent.join(table_names))

for table_name in table_names:
    result = cur.execute("PRAGMA table_info('%s')" % table_name).fetchall()
    column_names = list(zip(*result))[1]
    print (("\ncolumn names for %s:" % table_name)
           +newline_indent
           +(newline_indent.join(column_names)))


tables are:
Country
League
Match
Player
Player_Attributes
Team
Team_Attributes
sqlite_sequence

column names for Country:
id
name

column names for League:
id
country_id
name

column names for Match:
id
country_id
league_id
season
stage
date
match_api_id
home_team_api_id
away_team_api_id
home_team_goal
away_team_goal
home_player_X1
home_player_X2
home_player_X3
home_player_X4
home_player_X5
home_player_X6
home_player_X7
home_player_X8
home_player_X9
home_player_X10
home_player_X11
away_player_X1
away_player_X2
away_player_X3
away_player_X4
away_player_X5
away_player_X6
away_player_X7
away_player_X8
away_player_X9
away_player_X10
away_player_X11
home_player_Y1
home_player_Y2
home_player_Y3
home_player_Y4
home_player_Y5
home_player_Y6
home_player_Y7
home_player_Y8
home_player_Y9
home_player_Y10
home_player_Y11
away_player_Y1
away_player_Y2
away_player_Y3
away_player_Y4
away_player_Y5
away_player_Y6
away_player_Y7
away_player_Y8
away_player_Y9
away_player_Y10
away_player_Y11
home_player_

## Which team scored the most points when playing at home?  

In [32]:
get_match = """
SELECT
    M.country_id,
    Country.name,
    M.league_id,
    M.home_team_api_id,
    SUM(M.home_team_goal) AS TotGoals,
    Team.team_long_name,
    Team.team_short_name
FROM Match AS M JOIN Team ON M.home_team_api_id = Team.team_api_id
     JOIN Country ON M.country_id = Country.id
GROUP BY M.home_team_api_id
ORDER BY TotGoals DESC
LIMIT 10;
"""

#cur.execute(get_match)
#cur.fetchmany(10)
query1 = pd.read_sql_query(get_match, conn)
query1

,country_id,name,league_id,home_team_api_id,TotGoals,team_long_name,team_short_name
0,21518,Spain,21518,8633,505,Real Madrid CF,REA
1,21518,Spain,21518,8634,495,FC Barcelona,BAR
2,19694,Scotland,19694,9925,389,Celtic,CEL
3,7809,Germany,7809,9823,382,FC Bayern Munich,BMU
4,13274,Netherlands,13274,8640,370,PSV,PSV
5,1729,England,1729,8456,365,Manchester City,MCI
6,13274,Netherlands,13274,8593,360,Ajax,AJA
7,24558,Switzerland,24558,9931,344,FC Basel,BAS
8,1729,England,1729,10260,338,Manchester United,MUN
9,1729,England,1729,8455,333,Chelsea,CHE


## Did this team also score the most points when playing away? 

In [33]:
get_match_away = """
SELECT
    M.country_id,
    Country.name,
    M.league_id,
    M.away_team_api_id,
    SUM(M.away_team_goal) AS TotGoals,
    Team.team_long_name,
    Team.team_short_name
FROM Match AS M JOIN Team ON M.away_team_api_id = Team.team_api_id
     JOIN Country ON M.country_id = Country.id
GROUP BY M.away_team_api_id
ORDER BY TotGoals DESC
LIMIT 10;
"""

#cur.execute(get_match)
#cur.fetchmany(10)
query2 = pd.read_sql_query(get_match_away, conn)
query2

,country_id,name,league_id,away_team_api_id,TotGoals,team_long_name,team_short_name
0,21518,Spain,21518,8634,354,FC Barcelona,BAR
1,21518,Spain,21518,8633,338,Real Madrid CF,REA
2,19694,Scotland,19694,9925,306,Celtic,CEL
3,13274,Netherlands,13274,8593,287,Ajax,AJA
4,13274,Netherlands,13274,8640,282,PSV,PSV
5,24558,Switzerland,24558,9931,275,FC Basel,BAS
6,7809,Germany,7809,9823,271,FC Bayern Munich,BMU
7,1729,England,1729,9825,267,Arsenal,ARS
8,7809,Germany,7809,9789,253,Borussia Dortmund,DOR
9,1729,England,1729,8455,250,Chelsea,CHE


## How many matches resulted in a tie? 

In [41]:
get_tied_matches = """
SELECT
    COUNT(Match.id) AS TiedMatches
FROM Match
WHERE home_team_goal == away_team_goal;
"""
#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query3 = pd.read_sql_query(get_tied_matches, conn)
query3

,TiedMatches
0,6596


## How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [51]:
get_player_names = """
SELECT
    Count(id)
FROM Player
WHERE player_name LIKE '_% smith';
"""
#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query4a = pd.read_sql_query(get_player_names, conn)
query4a

,Count(id)
0,15


In [49]:
get_player_names2 = """
SELECT
    *
FROM Player
WHERE player_name LIKE '%smith%'
Limit 20;
"""
#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query4b = pd.read_sql_query(get_player_names2, conn)
query4b

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,129,160447,Adam Smith,190885,1991-04-29 00:00:00,180.34,179
1,263,30248,Alan Smith,13739,1980-10-28 00:00:00,180.34,179
2,1348,515338,Brad Smith,210389,1994-04-09 00:00:00,175.26,154
3,1513,315888,Cameron Smith,208691,1995-08-24 00:00:00,175.26,134
4,1729,34656,Chris Smith,155881,1986-03-05 00:00:00,187.96,190
5,2019,279105,Daan Smith,204389,1992-03-18 00:00:00,193.04,174
6,2416,264839,David Smith,206564,1993-03-01 00:00:00,180.34,165
7,3693,23999,Gary Naysmith,4190,1978-11-16 00:00:00,177.80,161
8,3949,91790,Gordon Smith,198372,1991-02-14 00:00:00,190.50,176
9,3958,24984,Graeme Smith,139539,1982-10-03 00:00:00,187.96,181


In [52]:
get_player_names3 = """
SELECT
    Count(id)
FROM Player
WHERE player_name LIKE '%smith%';
"""
#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query4b2 = pd.read_sql_query(get_player_names3, conn)
query4b2

,Count(id)
0,18


## What was the median tie score? Use the value determined in the previous question for the number of tie games. 

In [55]:
get_tied_matches2 = """
SELECT
    COUNT(Match.id) AS TiedMatches,
    home_team_goal,
    away_team_goal
    
FROM Match
WHERE home_team_goal == away_team_goal
GROUP BY home_team_goal;
"""
#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query5 = pd.read_sql_query(get_tied_matches2, conn)
query5

,TiedMatches,home_team_goal,away_team_goal
0,1978,0,0
1,3014,1,1
2,1310,2,2
3,264,3,3
4,27,4,4
5,2,5,5
6,1,6,6


The total number of tied mataches = <b>6,596</b>. The median is where <b>3,298</b><i>th</i> match is after the matches are arranged in ascending order of scores.  That would be a score of <b><i>"1-1"</i></b>

In [ ]:
get_tied_matches2 = """
SELECT
    COUNT(Match.id) AS TiedMatches,
    home_team_goal,
    away_team_goal    
FROM Match
WHERE home_team_goal == away_team_goal
GROUP BY home_team_goal;
"""
#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query5 = pd.read_sql_query(get_tied_matches2, conn)
query5

## What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [72]:
get_player1 = """
SELECT
    *   
FROM Player;
"""
get_player2 = """
SELECT
    *   
FROM Player_Attributes;
"""

#WHERE home_team_goal == away_team_goal;
#cur.execute(get_match)
#cur.fetchmany(10)
query6a = pd.read_sql_query(get_player1, conn)
query6b = pd.read_sql_query(get_player2, conn)
print(f"Number of players from Player table = {query6a.shape[0]}")
print(f"Number of players from Player_Attributes table = {query6b.shape[0]}")

Number of players from Player table = 11060
Number of players from Player_Attributes table = 183978


In [76]:
get_players_RightFoot = """
SELECT
    player_api_id,
    AVG(preferred_foot == 'right') AS Pref
FROM Player_Attributes
WHERE preferred_foot == 'right'
GROUP BY player_api_id
ORDER BY Pref DESC;
"""
get_ALL_Players = """
SELECT
    COUNT(id)
FROM Players;
"""
#WHERE preferred_foot == 'right'

query6c = pd.read_sql_query(get_players_RightFoot, conn)
players_right_foot = query6c.shape[0]
print(f"Total number of players = {query6a.shape[0]}")
print(f"Number of players who prefer right foot are {players_right_foot}")
print(f"Percentage of players who prefer right foot are {players_right_foot/query6a.shape[0]:.4f}")
query6c

Total number of players = 11060
Number of players who prefer right foot are 8979
Percentage of players who prefer right foot are 0.8118


,player_api_id,Pref
0,2625,1.0
1,2752,1.0
2,2770,1.0
3,2796,1.0
4,2802,1.0
...,...,...
8974,743720,1.0
8975,746419,1.0
8976,748432,1.0
8977,750435,1.0
